In [27]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import optuna.integration.lightgbm as lgb
from datetime import datetime

In [28]:
train = pd.read_csv('..\\kaggle_data\\train.csv')
test = pd.read_csv('..\\kaggle_data\\test.csv')

In [29]:
cat_features = [
    "cat0", "cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7", 
    "cat8", "cat9"
]

cont_features = [
    "cont0", "cont1", "cont2", "cont3", "cont4",
    "cont5", "cont6", "cont7", "cont8", "cont9", "cont10", 
    "cont11", "cont12", "cont13"
]
alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
alpha_conversion = {}
for index in range(0, len(alpha)):
    alpha_conversion[alpha[index]] = index / 29
alpha_conversion

for cat in cat_features:
    train[cat].replace(alpha_conversion, inplace=True)
    train[cat] = train[cat].astype('float')
train = train.drop(['id'], axis=1)
print(train.head())

for cat in cat_features:
    test[cat].replace(alpha_conversion, inplace=True)
    test[cat] = test[cat].astype('float')
print(test.head())

       cat0      cat1  cat2      cat3      cat4      cat5  cat6      cat7  \
0  0.000000  0.034483   0.0  0.000000  0.034483  0.103448   0.0  0.137931   
1  0.034483  0.000000   0.0  0.000000  0.034483  0.034483   0.0  0.137931   
2  0.000000  0.000000   0.0  0.068966  0.034483  0.103448   0.0  0.034483   
3  0.000000  0.000000   0.0  0.068966  0.034483  0.103448   0.0  0.137931   
4  0.000000  0.034483   0.0  0.000000  0.034483  0.034483   0.0  0.137931   

       cat8      cat9  ...     cont5     cont6     cont7     cont8     cont9  \
0  0.068966  0.275862  ...  0.881122  0.421650  0.741413  0.895799  0.802461   
1  0.000000  0.172414  ...  0.440011  0.346230  0.278495  0.593413  0.546056   
2  0.068966  0.448276  ...  0.914155  0.369602  0.832564  0.865620  0.825251   
3  0.206897  0.344828  ...  0.934138  0.578930  0.407313  0.868099  0.794402   
4  0.068966  0.172414  ...  0.382600  0.705940  0.325193  0.440967  0.462146   

     cont10    cont11    cont12    cont13    target  
0 

In [30]:
columns = [col for col in train.columns.to_list() if col not in ['id','target']]
data=train[columns]
target=train['target']
train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.15,random_state=42)
test_x = test.drop(['id'], axis=1)

In [31]:
dtrain = lgb.Dataset(train_x, label=train_y)
dval = lgb.Dataset(val_x, label=val_y)

params = {
    #"objective": "binary",
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

model = lgb.train(
    params, dtrain, valid_sets=[dtrain, dval], verbose_eval=1000, early_stopping_rounds=100, time_budget=(60*60)*2, model_dir='N:\\KaggleCompete\\boosters', show_progress_bar=False
)

prediction = model.predict(test_x, num_iteration=model.best_iteration)

best_params = model.params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))
    
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
submission = pd.DataFrame({'id':test.id,
          'target':prediction})
submission.to_csv(f'..\\kaggle_data\\{dt_string}_sub.csv', index=False)

[I 2021-02-26 17:21:21,187] A new study created in memory with name: no-name-fcbab382-0472-4ab8-bf3b-21314a7d7141


Training until validation scores don't improve for 100 rounds


[I 2021-02-26 17:21:23,322] The booster of trial#0 was saved as N:\KaggleCompete\boosters\0.pkl.
[I 2021-02-26 17:21:23,324] Trial 0 finished with value: 0.8457063116776244 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.8457063116776244.


Early stopping, best iteration is:
[289]	valid_0's rmse: 0.822572	valid_1's rmse: 0.845706
Training until validation scores don't improve for 100 rounds


[I 2021-02-26 17:21:25,476] The booster of trial#1 was saved as N:\KaggleCompete\boosters\1.pkl.
[I 2021-02-26 17:21:25,479] Trial 1 finished with value: 0.8454288096094538 and parameters: {'feature_fraction': 0.6}. Best is trial 1 with value: 0.8454288096094538.


...

Early stopping, best iteration is:
[865]	valid_0's rmse: 0.827495	valid_1's rmse: 0.84414
Training until validation scores don't improve for 100 rounds


[I 2021-02-26 17:24:54,334] The booster of trial#63 was saved as N:\KaggleCompete\boosters\63.pkl.
[I 2021-02-26 17:24:54,341] Trial 63 finished with value: 0.8444569286096317 and parameters: {'min_child_samples': 10}. Best is trial 62 with value: 0.8441396154385543.


Early stopping, best iteration is:
[761]	valid_0's rmse: 0.829218	valid_1's rmse: 0.844457
Training until validation scores don't improve for 100 rounds


[I 2021-02-26 17:24:58,273] The booster of trial#64 was saved as N:\KaggleCompete\boosters\64.pkl.
[I 2021-02-26 17:24:58,281] Trial 64 finished with value: 0.8441220140509186 and parameters: {'min_child_samples': 100}. Best is trial 64 with value: 0.8441220140509186.


Early stopping, best iteration is:
[751]	valid_0's rmse: 0.830016	valid_1's rmse: 0.844122
Best params: {'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 1.3600093060434386e-08, 'lambda_l2': 0.232367604889057, 'num_leaves': 10, 'feature_fraction': 0.44800000000000006, 'bagging_fraction': 0.8790593955789898, 'bagging_freq': 3, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
  Params: 
    metric: rmse
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 1.3600093060434386e-08
    lambda_l2: 0.232367604889057
    num_leaves: 10
    feature_fraction: 0.44800000000000006
    bagging_fraction: 0.8790593955789898
    bagging_freq: 3
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 100


In [23]:
test

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,0.0,0.034483,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.137931,...,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,0.0,0.034483,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.068966,...,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,0.0,0.034483,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.068966,...,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,0.0,0.000000,0.034483,0.000000,0.034483,0.103448,0.0,0.137931,0.137931,...,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,0.0,0.034483,0.000000,0.000000,0.034483,0.034483,0.0,0.137931,0.137931,...,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499987,0.0,0.000000,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.206897,...,0.277365,0.963678,0.240482,0.686462,0.915165,0.848878,0.459598,0.590327,0.864873,0.425258
199996,499990,0.0,0.000000,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.137931,...,0.523174,0.232072,0.363421,0.694092,0.137002,0.319465,0.364527,0.388908,0.664357,0.224215
199997,499991,0.0,0.000000,0.000000,0.068966,0.034483,0.103448,0.0,0.137931,0.068966,...,0.517103,0.432927,0.811876,0.328398,0.496017,0.538779,0.466338,0.643869,0.749590,0.457702
199998,499994,0.0,0.034483,0.000000,0.000000,0.034483,0.103448,0.0,0.137931,0.068966,...,0.279153,0.837712,0.680886,0.534439,0.501588,0.809053,0.631704,0.766426,0.937139,0.796304


In [24]:
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
submission = pd.DataFrame({'id':test.id,
          'target':prediction})
submission.to_csv(f'..\\kaggle_data\\{dt_string}_sub.csv', index=False)

In [21]:
best_params = model.params
print("Best params:", best_params)
#print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

Best params: {'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 9.4016532152438, 'lambda_l2': 0.006560267182579874, 'num_leaves': 10, 'feature_fraction': 0.4, 'bagging_fraction': 0.7091814665169984, 'bagging_freq': 7, 'min_child_samples': 25, 'num_iterations': 1000, 'early_stopping_round': 100}
  Params: 
    metric: rmse
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 9.4016532152438
    lambda_l2: 0.006560267182579874
    num_leaves: 10
    feature_fraction: 0.4
    bagging_fraction: 0.7091814665169984
    bagging_freq: 7
    min_child_samples: 25
    num_iterations: 1000
    early_stopping_round: 100


In [ ]:
model.predict(X_test)